In [38]:
import pandas as pd
import numpy as np
from SPARQLWrapper import SPARQLWrapper, JSON

In [129]:
EC2_URI = 'ec2-52-15-107-252.us-east-2.compute.amazonaws.com'

In [148]:
# grep "<http://dbkwik.webdatacommons.org/Phineas_and_Ferb_Fano/" dbkwik.nt > phineas.nt
# :%s/> .$/>/g

# REPEAT COMICS

# vocal.nt
# call.nt

pd.options.display.max_colwidth = 100
df = pd.read_csv('comics.nt', sep='\s+', names=['s', 'p', 'o'])

In [149]:
wiki = 'http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik'
ontologies = df[df['o'].str.contains(wiki + '/ontology')]['o'].unique()
properties = df[df['p'].str.contains(wiki + '/property')]['p'].unique()
resources = df[df['s'].str.contains(wiki + '/resource')]['s'].unique()

In [151]:
print(len(properties))
print(len(resources))
print(len(ontologies))
print(ontologies)

1480
131933
60
['<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Image>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Agent>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Company>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Organisation>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Comic>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Work>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/WrittenWork>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Cartoon>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/HollywoodCartoon>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/ComicStrip>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/ComicsCharacter>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/FictionalCharacter>'
 '<http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wi

In [158]:
def get_class_count(ontology):
    sparql = SPARQLWrapper("http://" + EC2_URI + ":3030/dbkwik/query")
    query = """
        SELECT (count(?s) as ?count)        
        WHERE { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> """ + ontology + """ }
    """
    print(query)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    for result in results["results"]["bindings"]:
        return result['count']['value']

class_frequencies = {}
for index in range(len(ontologies)):
    class_frequencies[ontologies[index]] = get_class_count(ontologies[index])

print(class_frequencies)


        SELECT (count(?s) as ?count)        
        WHERE { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Image> }
    

        SELECT (count(?s) as ?count)        
        WHERE { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Agent> }
    

        SELECT (count(?s) as ?count)        
        WHERE { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Company> }
    

        SELECT (count(?s) as ?count)        
        WHERE { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Organisation> }
    

        SELECT (count(?s) as ?count)        
        WHERE { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Comic> }
    

        SELECT (count(?s) as ?


        SELECT (count(?s) as ?count)        
        WHERE { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/CareerStation> }
    

        SELECT (count(?s) as ?count)        
        WHERE { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/TimePeriod> }
    

        SELECT (count(?s) as ?count)        
        WHERE { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Writer> }
    

        SELECT (count(?s) as ?count)        
        WHERE { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/PersonFunction> }
    

        SELECT (count(?s) as ?count)        
        WHERE { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Award> }
    

        SELECT (

In [159]:
def get_property_count_by_class(prop, classes, class_frequencies):
    sparql = SPARQLWrapper("http://" + EC2_URI + ":3030/dbkwik/query")
    query = """
        SELECT ?class (count(?class) as ?count) WHERE { 
            VALUES ?class { """ + classes + """ }
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> ?class .
            {
                select distinct ?s {
                    {?s """ + prop + """ ?o}
                    UNION { ?o """ + prop + """ ?s }
                }
            }            
        } group by ?class
    """
    print(query)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    output = {}
    for result in results["results"]["bindings"]:
        if 'class' in result:
            output['<' + result['class']['value'] + '>'] = int(result['count']['value']) / int(class_frequencies['<' + result['class']['value'] + '>'])
    return output
        
classes = ' '.join(ontologies)
property_frequencies = {}

length = len(properties)
for index in range(length):
    property_frequencies[properties[index]] = get_property_count_by_class(properties[index], classes, class_frequencies)
    print(properties[index], index, length, ' DONE')
    break

# Save to file    
with open("props.nt", "w") as file:
    for prop in property_frequencies:
        for ont in property_frequencies[prop]:
            file.write(prop + ' ' + ont + ' "' + str(property_frequencies[prop][ont]) + '"^^<http://www.w3.org/2001/XMLSchema#double> . \n')
print('File Saved')


        SELECT ?class (count(?class) as ?count) WHERE { 
            VALUES ?class { <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Image> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Agent> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Company> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Organisation> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Comic> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Work> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/WrittenWork> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/Cartoon> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/HollywoodCartoon> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/ComicStrip> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/ComicsCharacter> <http://dbkwik.webdatacommons.org/Hey_Kids_Comics_Wik/ontology/FictionalCharacter> <http://dbkwik.webd

In [147]:
property_frequencies = {}
def get_property_count(prop, resource_count):
    sparql = SPARQLWrapper("http://" + EC2_URI + ":3030/dbkwik/query")
    query = """
        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s """ + prop + """ ?o .            
        }
    """
    print(query)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    for result in results["results"]["bindings"]:
        return int(result['count']['value']) / resource_count
    return 0

for index in range(len(properties)):
    property_frequencies[properties[index]] = get_property_count(properties[index], len(resources))
    print(properties[index], property_frequencies[properties[index]])

# Save to File    
with open("props_rank.nt", "w") as file:
    count = 0
    length = len(property_frequencies)
    for prop in property_frequencies:
        file.write(prop + ' <http://purl.org/voc/vrank#proprank> ' + '"' + str(property_frequencies[prop]) + '"^^<http://www.w3.org/2001/XMLSchema#double> . \n')
        print(count, length)
        count++
print('File Saved')


        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Baahubali_Wik/property/age> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Baahubali_Wik/property/age> 0.11538461538461539

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Baahubali_Wik/property/aliases> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Baahubali_Wik/property/aliases> 0.1346153846153846

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Baahubali_Wik/property/enemies> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Baahubali_Wik/property/enemies> 0.17307692307692307

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Baahubali_Wik/property/eyeColour> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Baahubali_Wik/property/eyeColour> 0.192

<http://dbkwik.webdatacommons.org/Baahubali_Wik/property/author> 0.019230769230769232

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Baahubali_Wik/property/country> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Baahubali_Wik/property/country> 0.019230769230769232

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Baahubali_Wik/property/pages> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Baahubali_Wik/property/pages> 0.019230769230769232

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Baahubali_Wik/property/published> ?o .            
        }
    
<http://dbkwik.webdatacommons.org/Baahubali_Wik/property/published> 0.019230769230769232

        SELECT (COUNT(DISTINCT ?s) as ?count) WHERE { 
            ?s <http://dbkwik.webdatacommons.org/Baahubali_Wik/property/publisher> ?o .   